In [367]:
import networkx as nx
import scripts.read_graph as rg


In [368]:
def read(path):

    MG = nx.read_gexf(path)

    for node in MG.nodes(data=True):
        for key, value in MG.edge[node[0]].items():
            for k, v in value.items():
                if type(v['attr']) == str:
                    MG.edge[node[0]][key][k]['attr'] = ast.literal_eval(v['attr'])

    for node in MG.nodes(data=True):
        if isinstance(MG.node[node[0]]['location'], str):
            MG.node[node[0]]['location'] = ast.literal_eval(MG.node[node[0]]['location'])
            if not isinstance(MG.node[node[0]]['location'], dict):
                print('NAO FUNCIONA')
                break
        if len(node[1]) == 3:
            if type(MG.node[node[0]]['viagens']) == str:
                MG.node[node[0]]['viagens'] = ast.literal_eval(MG.node[node[0]]['viagens'])

    return MG

In [369]:
MG = read('graphs/sp_final.gexf')

Não deixar o fera ficar entrando e saindo de busao

In [404]:
class Walker():
    
    def __init__(self, graph, stop_id, travel_time, current_time):
        self.MG = graph
        self.stop_id = stop_id
        self.total_time_travel = travel_time * 60 # minutos para segundos
        self.traveled_time = 0
        self.visited_set = []
        self.tentative_value = {}
        self.current_time = current_time * 60 * 60 # hora para segundo
        
        self.calculate_location = lambda x: (MG.node[x]['location']['lat'], MG.node[x]['location']['long'])# tuple
        self.current_location = self.calculate_location(stop_id)
        
    def isochrone(self):
        i=0
        while 1:
            print('\n{}: '.format(i))
            self.find_neighbours()

            minimum = -1
            for key, values in self.tentative_value.items():
                
                if key not in self.visited_set:
                    
                    if minimum == -1:
                        minimum = values['travel_time']
                        min_key = key
                    elif values['travel_time'] > minimum:
                        minimum = values['travel_time']
                        min_key = key

            self.stop_id = min_key

            self.traveled_time = self.traveled_time + minimum
            
            print('Busao:', self.tentative_value[min_key]['linha'])
            print('Traveled Time: ',self.traveled_time)
            if self.traveled_time > self.total_time_travel:
                break
            i=i+1
    
    def find_neighbours(self):
        """Given a stop, find neighbours and calculate travel time. 
        Two types of neighbours: connectionable and walkable
        """
        print('StopId: ', self.stop_id)
        # get connection neighbours
        linha_atual = 0
        if len(self.visited_set) > 0:
            print('Linha Atual:', self.tentative_value[self.stop_id]['linha'])
            linha_atual = self.tentative_value[self.stop_id]['linha']
        for conn_neigh_key, conn_neigh_value in self.MG.node[self.stop_id]['viagens'].items():
            
            #print('TripId', conn_neigh_key)
            # get next travel time
                
            travel_time =  (min(conn_neigh_value,
                              key=lambda x: abs(x-self.current_time) if x-self.current_time<=0 else False) -
                            self.current_time)
            
            if conn_neigh_key == linha_atual:
                print(travel_time)
            # have to be greater than current_time
            if travel_time < 0:
                continue
            
            # get time to next stop  
            output = getpath(self.MG.edge[self.stop_id], conn_neigh_key)
            if output == None:
                continue
            stop_id_next, trip_key, *rest = output
            
            if stop_id_next in self.visited_set:
                continue
            #print('StopIdNext: ',stop_id_next)
            travel_time = travel_time + self.MG.edge[self.stop_id][stop_id_next][trip_key]['attr']['tempo']
            
            # check travel time to next stop and replace if smaller
            if stop_id_next in self.tentative_value:
                if travel_time < self.tentative_value[stop_id_next]['travel_time']:
                    self.tentative_value[stop_id_next] = {'travel_time': travel_time,
                                                          'trip_key': trip_key,
                                                          'previous_stop': self.stop_id,
                                                        'linha': conn_neigh_key}
            else:
                self.tentative_value[stop_id_next] = {'travel_time': travel_time,
                                                      'trip_key': trip_key,
                                                      'previous_stop': self.stop_id,
                                                     'linha': conn_neigh_key}
   
            #print(self.tentative_value)
    
        self.visited_set.append(self.stop_id)
        
        # get walkable neighbours
        

In [405]:
node_id = '706304'
walker = Walker(MG, node_id, travel_time=60, current_time=21)
walker.isochrone()
print(walker.stop_id)



0: 
StopId:  706304
Busao: 874T-10-0
Traveled Time:  124

1: 
StopId:  70016916
Linha Atual: 874T-10-0
124
Busao: 874T-10-0
Traveled Time:  355

2: 
StopId:  70015949
Linha Atual: 874T-10-0
231
Busao: 917H-10-1
Traveled Time:  715

3: 
StopId:  70016918
Linha Atual: 917H-10-1
360
Busao: 917M-10-1
Traveled Time:  895

4: 
StopId:  260016920
Linha Atual: 917M-10-1
180
Busao: 805L-10-0
Traveled Time:  1341

5: 
StopId:  2600690
Linha Atual: 805L-10-0
446
Busao: 875P-10-1
Traveled Time:  1615

6: 
StopId:  260016732
Linha Atual: 875P-10-1
274
Busao: 175P-10-1
Traveled Time:  1825

7: 
StopId:  260016733
Linha Atual: 175P-10-1
210
Busao: 719P-10-1
Traveled Time:  2275

8: 
StopId:  440014681
Linha Atual: 719P-10-1
450
Busao: 909T-10-1
Traveled Time:  2545

9: 
StopId:  440014682
Linha Atual: 909T-10-1
270
Busao: 975A-10-1
Traveled Time:  2784

10: 
StopId:  260016735
Linha Atual: 975A-10-1
239
Busao: 875H-10-1
Traveled Time:  3742
610012509


In [377]:
print(MG.node['610012509'])


{'viagens': {'975A-10-0': [23730, 27330, 28050, 28650, 29250, 29850, 30450, 31050, 31650, 32250, 32850, 33450, 34050, 35250, 36450, 37650, 38850, 39750, 40650, 41550, 42750, 43950, 45150, 46350, 47250, 48150, 49050, 49950, 50850, 51750, 52650, 53550, 54450, 55350, 56250, 57150, 58050, 58950, 59850, 60750, 61650, 62550, 63450, 64350, 65250, 66150, 67050, 67950, 68850, 69750, 70650, 71550, 73350, 75150, 76350, 77550, 78750, 79950, 81150, 82350, 83250, 84150, 85050, 85950, 87150, 88350, 89550, 91350, 6810], '875H-10-1': [24778, 28378, 29098, 29458, 29818, 30178, 30538, 30898, 31258, 31618, 31978, 32338, 32638, 32938, 33238, 33538, 33838, 34138, 34438, 34738, 35038, 35338, 35638, 36058, 36478, 36898, 37318, 37738, 38158, 38578, 38998, 39418, 39778, 40138, 40498, 40858, 41218, 41578, 41938, 42298, 42658, 43018, 43498, 43978, 44458, 44938, 45418, 45898, 46378, 46858, 47578, 48298, 49018, 49738, 50458, 51058, 51658, 52258, 52858, 53458, 54058, 54658, 55258, 55858, 56458, 57058, 57658, 58378, 

In [237]:
def getpath(nested_dict, value, prepath=()):
    for k, v in nested_dict.items():
        path = prepath + (k,)
        if v == value: # found value
            return path
        elif hasattr(v, 'items'): # v is a dict
            p = getpath(v, value, path) # recursive call
            if p is not None:
                return p

In [106]:
import folium
COORD = (-23.542225, -46.635807) # sp coordinates
mapa = folium.Map(location=COORD, zoom_start=11)

for i, stop in enumerate(MG.nodes(data=True)[:100]):
    if i == 0:
        color = '#ff0000'
        radius=5
    else:
        color = '#45647d'
        radius=1
    
    folium.RegularPolygonMarker(
        location = [stop[1]['location']['lat'],stop[1]['location']['long']],
        fill_color=color,
        number_of_sides=100,
        radius=radius,
        popup='Lat: {}, Long: {} \n id: {}'.format(stop[1]['location']['lat'],stop[1]['location']['long'],
                                                  stop[0])
        ).add_to(mapa)
mapa